In [3]:
# 5.1 선형 SVM 분류

In [4]:
# 5.1.1 소프트 마진 분류
# 도로의 폭을 가능한 한 넓게 유지하는 것과 마진 오류 사이에 적절한 균형을 잡아 분류하는 모델.

In [5]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [6]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

In [7]:
# 5.2 비선형 SVM 분류

In [8]:
# 선형 커널을 사용하는 LinearSVC 클래스

from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15)
polynomal_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

polynomal_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [9]:
# 다항 커널을 사용하는 SVM 분류기 (커널 트릭을 사용하여)

from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [10]:
# 5.2.2 유사도 특성

In [11]:
# 식 5-1 가우시안 RBF

# l 은 랜드마크 지점.

$\phi_r (\mathbf x, l) = exp(-\gamma {\lVert {\mathbf x - l} \rVert}^2)$

In [12]:
# 5.2.3 가우시안 RBF 커널

In [13]:
# 가우시안 RBF 커널을 사용하는 SVM 분류기 (역시 커널 트릭을 사용하여)

rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

In [14]:
# 5.2.4 계산 복잡도

# LinearSVC - 커널 트릭 지원 x, O(n * m)
# SVC - 커널 트릭 지원 O, O(m^2 * n) ~ O(m^3 * n). 특성의 개수는 희소특성의 경우에 잘 확장 됨. 즉, 성능이 샘플이 가진 0이 아닌 특성의 평균 수에 거의 비례.

In [15]:
# 5.3 SVM 회귀

In [16]:
# 일정한 마진 오류 안에서 두 클래스 간의 도로 폭이 가능한 한 최대가 되도록 하는 대신, 제한된 마진 오류 안에서 도로 안에 가능한 한 많은 샘플이 들어가도록 학습.

In [18]:
# 선형 SVM 회귀
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5)

In [20]:
# 비선형 SVM 회귀
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)


SVR(C=100, degree=2, kernel='poly')

In [21]:
# 5.4 SVM 이론

In [22]:
# 5.4.1 결정 함수와 예측

In [23]:
# 식 5-2 선형 SVM 분류기의 예측

$\hat {y} = 
\begin{cases} 
    0, & if \quad w^Tx + b < 0 \newline 
    1, & if \quad w^Tx + b \ge 0
\end{cases}$

In [26]:
# 5.4.2 목적 함수

In [27]:
# 가중치 벡터 w가 작을 수록 마진은 커진다. 

In [28]:
# 식 5-3 하드 마진 선형 SVM 분류기의 목적 함수

$$\begin{align} 
& \underset {w, b} {minimize} \; {1 \over 2} \mathbf {w}^T \mathbf {w} \\
& condition : when \; i = 1, 2, \dots, m, \quad t^{(i)}(\mathbf {w}^T \mathbf {x}^{(i)} + b) \ge 1
\end{align}$$

In [29]:
# 식 5-4 소프트 마진 선형 SVM 분류기의 목적 함수

$$\begin{align} 
& \underset {w, b, \zeta} {minimize} \; {1 \over 2} \mathbf {w}^T \mathbf {w} + C \sum_{i=1}^m \zeta^{(i)}\\
& condition : when \; i = 1, 2, \dots, m, \quad t^{(i)}(\mathbf {w}^T \mathbf {x}^{(i)} + b) \ge 1 - \zeta^{(i)} \quad and \quad \zeta^{(i)} \ge 0
\end{align}$$

In [31]:
# 5.4.3 콰드라틱 프로그래밍

# 하드 마진과 소프트 마진 문제는 모두 선형적인 제약 조건이 있는 볼록 함수의 이차 최적화 문제.
# 이런 문제를 QP, 쿼드라틱 프로그래밍 문제라고 함.

In [32]:
# 식 5-5 QP 문제

$$\begin{align} 
& \underset {\mathbf p} {minimize} \; {1 \over 2} \mathbf {p}^T \mathbf {H} \mathbf {p} + \mathbf {f}^T \mathbf {p}\\
& condition : \; \mathbf {A} \mathbf {p} \le \mathbf {b}\\
& 여기서 
\begin{cases} 
    \mathbf {p} 는 \; n_p 차원의 \; 벡터 (n_p = 모델 \; 파리미터 \; 수) \\ 
    \mathbf {H} 는 \; n_p \times n_p \; 크기 \; 행렬 \\
    \mathbf {f} 는 \; n_p 차원의 \; 벡터 \\
    \mathbf {A} 는 \; n_c \times n_p \; 크기 \; 행렬 (n_c = 제약 \; 수) \\
    \mathbf {b} 는 \; n_c 차원의 \; 벡터
\end{cases} 
\end{align}$$

In [33]:
# 하드 마진을 갖는 선형 SVM 분류기의 목적 함수 검증

# n_p = n + 1. n 은 특성 수 (편향 때문에 +1이 추가되었음.)
# n_c = m. m 은 훈련 샘플 수
# H 는 n_p * n+p 크기이고, 왼쪽 맨 위의 원소가 0 (편향을 제외하기 위해)인 것을 제외하고는 단위 행렬.
# f = 0. 모두 0으로 채워진 n_p 차원의 벡터
# b = 1. 모두 1으로 채워진 n_c 차원의 벡터
# a^(i) = -t^(i) * dot x^(i). 여기서 dot x^(i)는 편향을 위해 특성 dot x_0 = 1를 추가한 x^(i) 와 같다.

# 결과 벡터 p는 편향 b=p_0와 특성 가중치 w_i=p_i 를 담고 있다.

In [35]:
# 5.4.4 쌍대 문제

# 원 문제(primal problem)라는 제약이 있는 최적화 문제가 주어지면 쌍대 문제(dual problem)라고 하는, 깊게 관련된 다른 문제로 표현 가능.
# 일반적으로 쌍대 문제 해는 원 문제 해의 하한값이지만, 어떤 조건하에서는 원 문제와 똑같은 해를 제공. SVM 문제는 이 조건을 만족.

In [36]:
# 식 5-6 선형 SVM 목적 함수의 쌍대 형식

$$\begin{align} 
& \underset {\mathbf \alpha} {minimize} \; {1 \over 2} \sum_{i=1}^m \sum_{j=1}^m \alpha^{(i)} \alpha^{(j)} t^{(i)} t^{(j)}
{\mathbf {x}^{(i)}}^T \mathbf {x}^{(j)} - \sum_{i=1}^m \alpha^{(i)}\\
& condition : \quad when \quad i = 1, 2, \dots, m, \quad \alpha^{(i)} \ge 0
\end{align}$$

In [37]:
# 식 5-7 쌍대 문제에서 구한 해로 원 문제의 해 계산하기

$$\begin{align} 
& \hat {w} = \sum_{i=1}^m \hat {\alpha}^{(i)} t^{(i)} \mathbf {x}^{(i)}  \\
& \hat {b} = {1 \over n_s} \underset {\hat {\alpha}^{(i)} > 0} {\sum_{i=1}^m} (t^{(i)} - \hat {w}^T \mathbf {x}^{(i)})
\end{align}$$

In [38]:
# 훈련 샘플 수가 특성 개수보다 작을 때 원 문제보다 쌍대 문제를 푸는 것이 더 빠르다.
# 또한 원 문제에서 적용이 안되는 커널 트릭을 가능하게 한다.

In [39]:
# 5.4.5 커널 SVM

In [40]:
# 2차원 데이터셋에 2차 다항식 변환을 적용하고, 선형 SVM 분류기를 변환된 이 훈련 세트에 적용한다고 하자.

In [41]:
# 식 5-8. 2차 다항식 매핑

$\phi(\mathbf {x}) = \phi({x_1 \choose x_2}) =
\begin{pmatrix} x_1^2 \\ \sqrt {2} x_1 x_2 \\ x_2^2 \end{pmatrix}
$

In [42]:
# 변환된 벡터는 2차원이 아니고 3차원이 된다. 두 개의 2차원 벡터 a, b에 2차 다항식 매핑을 적용하고, 변환된 벡터로 점곱을 한다.

In [43]:
# 식 5-9 2차 다항식 매핑을 위한 커널 트릭

$\phi(\mathbf {a})^T \phi(\mathbf {b}) = 
{\begin{pmatrix} a_1^2 \\ \sqrt {2} a_1 a_2 \\ a_2^2 \end{pmatrix}}^T \begin{pmatrix} b_1^2 \\ \sqrt {2} b_1 b_2 \\ b_2^2 \end{pmatrix} =
a_1^2 b_1^2 + 2a_1b_1a_2b_2 + a_2^2 b_2^2 =
(a_1b_1 + a_2b_2)^2 = 
({a_1 \choose a_2}^T {b_1 \choose b_2})^2 =
(\mathbf {a}^T \mathbf {b})^2
$

In [44]:
# 결괏값을 구하기 위해 훈련 샘플을 어렵게 변환해서 선형 SVM 알고리즘을 적용하는 것과 위 식처럼 점곱을 제곱으로 바꾸는 것과 결과는 같다.
# 이렇게 계산량의 측면에서 훨씬 효율적으로 만들어 주는 기법이 커널 트릭이다.

In [45]:
# 머신러닝에서 커널은 변환 phi를 계산하지 않고 원래 벡터 a, b에 기반하여 점곱을 계산할 수 있는 함수이다. 

In [46]:
# 식 5-10. 일반적인 커널

$\begin{align} 
& linear: \quad K(a, b) = a^T b \\
& poly: \quad K(a, b) = (\gamma a^T b + r)^d \\
& gaussian \; RBF: \quad K(a, b) = exp(-\gamma {\parallel a - b \parallel}^2) \\
& sigmoid: \quad K(a, b) = tanh(\gamma a^T b + r)
\end{align}$

In [48]:
# 식 5-7은 선형 SVM 분류기일 경우 쌍대 문제를 풀어서 원 문제를 해결하는 방법을 알려준다.
# 그러나 커널 트릭을 사용하면 예측 식에서 phi(x^(i))를 포함해야 한다.
# 결국 hat {w}의 차원이 매우 크거나 무한한 phi(x^(i))와 같아져야 하므로, 이를 계산할 수 없다.
# 그럼에도 hat {w}를 모르는 채로 예측을 만들 수 있다.

# 식 5-7의 hat {w} 에 대한 식을 새로운 샘플 x^(n) 의 결정 함수에 적용해서 입력 벡터 간의 점곱으로만 된 식을 얻을 수 있다.
# 따라서 커널 트릭을 사용할 수 있고, 식 5-11이 된다.

In [49]:
# 식 5-11 커널 SVM으로 예측하기

$\begin{align} 
& h_{\hat {w} \hat{b}} (\phi(x^{(n)})) = 
\hat {w}^T \phi(x^{(n)}) + \hat {b} = 
(\sum_{i=1}^m \hat {\alpha}^{(i)} t^{(i)} \phi(x^{(i)}))^T \phi(x^{(n)}) + \hat {b} \\
& = \sum_{i=1}^m \hat {\alpha}^{(i)} t^{(i)} (\phi(x^{(i)})^T \phi(x^{(n)})) + \hat {b} \\
& = \underset {\hat {\alpha}^{(i)} > 0} {\sum_{i=1}^m} \hat {\alpha}^{(i)} t^{(i)} K(x^{(i)}, x^{(n)}) + \hat {b}
\end{align}$

In [51]:
# 서포트 벡터만 alpha^(i) != 0 이기 때문에, 예측을 만드는 데는 전체 샘플이 아니라 서포트 벡터와 새 입력 백터 x^(n) 간의 점곱만 계산하면 됨.

In [50]:
# 식 5-12 커널 트릭을 사용한 편향 계산

$\begin{align} 
& \hat {b} = 
{1 \over n_s} \underset {\hat {\alpha}^{(i)} > 0} {\sum_{i=1}^m} (t^{(i)} - \hat {w}^T \phi(x^{(i)})) =
{1 \over n_s} \underset {\hat {\alpha}^{(i)} > 0} {\sum_{i=1}^m} (t^{(i)} - (\sum_{j=1}^m \hat {\alpha}^{(j)} t^{(j)} \phi(x^{(j)}))^T \phi(x^{(i)})) \\
& = {1 \over n_s} \underset {\hat {\alpha}^{(i)} > 0} {\sum_{i=1}^m} (t^{(i)} - \underset {\hat {\alpha}^{(j)} > 0} {\sum_{j=1}^m} \hat {\alpha}^{(j)} t^{(j)} K(x^{(i)}, x^{(j)}))
\end{align}$

In [52]:
# 5.4.6 온라인 SVM

In [53]:
# 온라인 학습은 새로운 샘플이 생겼을 때 점진적으로 학습하는 것을 말한다.

# 온라인 SVM 분류기를 구현하는 한 가지 방법은, 원 문제로부터 유도된 식 5-13의 비용함수를 최소화 하기 위한 경사 하강법을 사용하는 것.
# 경사하강법은 QP 기반의 방법보다 훨씬 느리게 수렴한다.

In [54]:
# 식 5-13. 선형 SVM 분류기 비용 함수

$J(w, b) =
{1 \over 2} w^T w + C \sum_{i=1}^m max(0, 1 - t^{(i)} (w^T x^{(i)} + b)) $